# Notebook 01: Data Fabric & Graph Construction

## From Tabular to Graphs: Representing Nuclear Topology

**Learning Objective:** Understand how to represent the Chart of Nuclides as a graph for GNN training using real EXFOR data.

**Focus:** See how U-235 and Cl-35 are positioned in the nuclear landscape and how GNNs can learn relationships between them.

### Why Graphs?

Nuclear data has inherent structure:
- **Isotopes** are related by reactions: U-235 + n → U-236 (capture)
- **Neighborhoods matter**: Similar Z/A have similar physics
- **Topology encodes physics**: Chart of Nuclides = Graph!
- **Transfer learning**: U-235 (data-rich) can inform Cl-35 (data-sparse) through graph structure

This structure is **invisible** to classical ML but **natural** for GNNs.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from pathlib import Path

from nucml_next.data import NucmlDataset
from nucml_next.utils import ChartOfNuclides

# Verify EXFOR data exists
exfor_path = Path('../data/exfor_processed.parquet')
if not exfor_path.exists():
    raise FileNotFoundError(
        f"EXFOR data not found at {exfor_path}\n"
        "Please run: python scripts/ingest_exfor.py --exfor-root <path> --output data/exfor_processed.parquet"
    )

print("✓ Imports successful")
print("✓ EXFOR data found")

### Step 1: Build the Nuclear Graph

In [ ]:
# Load real EXFOR data in GRAPH mode
dataset = NucmlDataset(
    data_path='../data/exfor_processed.parquet',
    mode='graph'
)

# Get global graph representation
graph = dataset.graph_builder.build_global_graph()

print(f"Graph Statistics:")
print(f"  Nodes (isotopes): {graph.num_nodes}")
print(f"  Edges (reactions): {graph.num_edges}")
print(f"  Node features: {graph.x.shape[1]} (includes AME2020 enrichment)")
print(f"  Edge features: {graph.edge_attr.shape[1]}")

### Step 2: Visualize the Chart of Nuclides

In [ ]:
# Plot Chart of Nuclides with U-235 and Cl-35 highlighted
chart = ChartOfNuclides()

# Highlight our focus isotopes
fig, ax = chart.plot_chart(
    dataset.df[['Z', 'N']].drop_duplicates(),
    highlight_isotopes=[
        (92, 235),  # U-235 (well-understood, data-rich)
        (17, 35)    # Cl-35 (research interest, data-sparse)
    ]
)
plt.show()

print("\n🔍 Observation: The chart shows nuclear topology!")
print("   • U-235 (Z=92): Heavy actinide region (highlighted)")
print("     - Fissile isotope, critical for reactors")
print("     - Well-characterized, extensive EXFOR data")
print()
print("   • Cl-35 (Z=17): Light element region (highlighted)")
print("     - (n,p) reaction produces S-35 (medical isotope)")
print("     - Limited experimental data, active research")
print()
print("   • GNNs can learn this structure and TRANSFER knowledge:")
print("     - Physics principles learned from U-235 (data-rich)")
print("     - Applied to Cl-35 predictions (data-sparse)")
print("     - Graph connections enable information flow!")

### 🎓 Key Takeaway

> The Chart of Nuclides is a **graph**, not a table!
>
> **U-235** and **Cl-35** are connected through:
> - Shared nuclear physics (Q-values, thresholds, conservation laws)
> - Graph topology (neighboring isotopes, reaction channels)
> - **GNN message passing**: Information flows through the graph!
>
> With **8D node features** (including AME2020 mass excess and binding energy), 
> the graph encodes rich nuclear physics for the GNN to learn from.
>
> **Key Advantage**: Models trained on data-rich isotopes (U-235) can improve 
> predictions for data-sparse isotopes (Cl-35) through graph structure!

Continue to `02_GNN_Transformer_Training.ipynb` →